In [1]:
import polars as pl
import numpy as np
import pandas as pd
import os
import xlsxwriter
import plotly.graph_objects as go

In [9]:
# functions
def calc_voyage_duration_hours(df, design_speed):
    return df.with_columns((pl.col('distance') / design_speed).alias('voyage_duration_hours'))

def calc_voyage_duration_days(df):
    return df.with_columns((pl.col('voyage_duration_hours') / 24).alias('voyage_duration_days'))

def calc_fuel_consumption(df, sfoc):
    return df.with_columns((pl.col('voyage_duration_days')*sfoc).alias('fuel_consumption_ton'))

def calc_hours_to_days(df):
    return df.with_columns((pl.col('container_handling_duration_hours')/24).alias('container_handling_duration_days'))

def rounding_days(df):
    return df.with_columns((pl.col('total_duration_days') + 0.999999).cast(pl.Int32).alias('total_duration_days'))

def calc_annual_cost_ice():
    return Pinv_ice*(power**1.1)*(annuity_factor+annual_maint_cost_factor_ice)

def calc_voyage_energy(df, power):
    return df.with_columns((pl.col('voyage_duration_hours')*power).alias('voyage_energy'))

def calc_annual_cost_bat(df, Pinv_bat, annuity_factor):
    return df.with_columns((pl.col('voyage_energy')*Pinv_bat*annuity_factor).alias('annual_cost_bat'))

def convt_idr_to_eur(data):
    return data/17000

def calc_ice_fuel_cost(df, fuel_price_eur):
    return df.with_columns((pl.col('fuel_consumption_ton')*fuel_price_eur*17).alias('ice_fuel_cost'))

def calc_electric_cost(df, electric_price_eur):
    return df.with_columns((pl.col('voyage_energy')*electric_price_eur*17).alias('be_electric_cost'))

# Define the function to round and cast the column
def round_and_cast(df, column_names):
    for column in column_names:
        df = df.with_columns(pl.col(column).round().cast(pl.Int64))
    return df

# Function to convert int to string and add thousand separators
def format_int_to_string(col):
    return col.map_elements(lambda x: f"{x:,}")


# Assumptions
design_speed = 16 # knots
sfoc = 23 # ton/day
power = 6000 # kW
total_annual_trips_numbers = 24
lifetime = 25 # years
int_rate = 0.04
Pinv_ice = 251.2 # euro/kW
Pinv_bat = 460 # euro/kWh
Ccell_bat = 733 # Wh cell specific energy capacity 
annual_maint_cost_factor_ice = 0.0045 
annuity_factor = (((1+int_rate)**lifetime)*int_rate)/(((1+int_rate)**lifetime)-1)  
annual_cost_ice = calc_annual_cost_ice()
electric_price = 996.74 # per kWh 
electric_price_eur = convt_idr_to_eur(electric_price)
fuel_density = 860 # kg/m3
fuel_price = (1000/fuel_density)*18950 # per liter convert to ton https://daihatsu.co.id/tips-and-event/tips-sahabat/detail-content/ini-daftar-harga-bbm-hari-ini-dan-klasifikasi-jenisnya/
fuel_price_eur = convt_idr_to_eur(fuel_price)


print(fuel_price_eur)



1296.1696306429549


In [20]:
# Trayek t-10 data
# https://ppid.dephub.go.id/fileupload/informasi-berkala/20230512143919.SK_Jaringan_Trayek_Tol_Laut_TA._2023.pdf
df = pl.DataFrame(
    {
        'port_depart': ['Tj Perak', 'TIdore', 'Morotai', 'Galela', 'Maba', 'Weda'],
        'port_arrive': ['Tidore', 'Morotai', 'Galela', 'Maba', 'Weda', 'Tj Perak'],
        'distance': [1216, 156, 72, 144, 139, 1213],
        'container_handling_duration_hours': [8, 36, 87, 80, 91, 8]
    }
)

df = df.with_columns(
    pl.lit(annual_cost_ice).alias('annual_cost_ice')
)

df = (
    df
    .pipe(calc_voyage_duration_hours, design_speed).pipe(calc_voyage_duration_days)
    .pipe(calc_fuel_consumption, sfoc).pipe(calc_hours_to_days)
    .pipe(calc_voyage_energy, power).pipe(calc_annual_cost_bat, Pinv_bat, annuity_factor)
    .pipe(calc_ice_fuel_cost, fuel_price_eur).pipe(calc_electric_cost, electric_price_eur)
)

# Specify the columns to round and cast
columns_to_round_and_cast = ['annual_cost_ice', 'annual_cost_bat', 'ice_fuel_cost', 'be_electric_cost']

# Apply the rounding and casting function to the selected columns
df = round_and_cast(df, columns_to_round_and_cast)

# Add formatted columns for hover display

for col in columns_to_round_and_cast:
    df = df.with_columns(format_int_to_string(pl.col(col)).alias(f"{col}_str"))


df.write_excel('polars_df.xlsx')







shape: (6, 17)
┌───────────┬───────────┬──────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ port_depa ┆ port_arri ┆ distance ┆ container ┆ … ┆ annual_co ┆ annual_co ┆ ice_fuel_ ┆ be_electr │
│ rt        ┆ ve        ┆ ---      ┆ _handling ┆   ┆ st_ice_st ┆ st_bat_st ┆ cost_str  ┆ ic_cost_s │
│ ---       ┆ ---       ┆ i64      ┆ _duration ┆   ┆ r         ┆ r         ┆ ---       ┆ tr        │
│ str       ┆ str       ┆          ┆ _hour…    ┆   ┆ ---       ┆ ---       ┆ str       ┆ ---       │
│           ┆           ┆          ┆ ---       ┆   ┆ str       ┆ str       ┆           ┆ str       │
│           ┆           ┆          ┆ i64       ┆   ┆           ┆           ┆           ┆           │
╞═══════════╪═══════════╪══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ Tj Perak  ┆ Tidore    ┆ 1216     ┆ 8         ┆ … ┆ 246,463   ┆ 13,427,14 ┆ 1,604,874 ┆ 454,513   │
│           ┆           ┆          ┆           ┆   ┆           ┆ 9         ┆

In [37]:
voyage_df = df

# Define bars for the ICE and Battery Electric costs with distinct colors
fig = go.Figure(data=[
    go.Bar(name='ICE Annualized Investment Cost', x=voyage_df['port_arrive'], y=voyage_df['annual_cost_ice'], 
           hovertext=voyage_df['annual_cost_ice_str'], marker=dict(color='darkred'), offsetgroup=0),
    go.Bar(name='ICE Fuel Cost', x=voyage_df['port_arrive'], y=voyage_df['ice_fuel_cost'], 
           hovertext=voyage_df['ice_fuel_cost_str'], marker=dict(color='salmon'), offsetgroup=0, base=voyage_df['annual_cost_ice']),
    go.Bar(name='Battery Annualized Investment Cost', x=voyage_df['port_arrive'], y=voyage_df['annual_cost_bat'], 
           hovertext=voyage_df['annual_cost_bat_str'], marker=dict(color='darkblue'), offsetgroup=1),
    go.Bar(name='Battery Charging Cost', x=voyage_df['port_arrive'], y=voyage_df['be_electric_cost'], 
           hovertext=voyage_df['be_electric_cost_str'], marker=dict(color='DeepSkyBlue'), offsetgroup=1, base=voyage_df['annual_cost_bat'])
])

# Update the layout for a grouped bar plot
fig.update_layout(
    barmode='group',
    title='Annual Costs for ICE and Battery Electric Ships by Port of Arrival',
    xaxis=dict(title='Port of Arrival'),
    yaxis=dict(title='Annual costs in €/a'),
    legend=dict(
        title='Cost Components',
        x=1,
        xanchor='left',
        y=1,
        yanchor='top',
        bgcolor='rgba(255,255,255,0)',
        bordercolor='rgba(255,255,255,0)'
    ),
    legend_orientation="v",
    margin=dict(l=0, r=0, t=30, b=0)  # Adjust margins to fit the legend outside the view if needed
)



# Add custom hovertemplate for each trace

fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{data.name}: €%{hovertext}<extra></extra>",
    selector=dict(name='ICE Annualized Investment Cost')
)
fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{data.name}: €%{hovertext}<extra></extra>",
    selector=dict(name='ICE Fuel Cost')
)
fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{data.name}: €%{hovertext}<extra></extra>",
    selector=dict(name='Battery Annualized Investment Cost')
)
fig.update_traces(
    hovertemplate="<b>%{x}</b><br>%{data.name}: €%{hovertext}<extra></extra>",
    selector=dict(name='Battery Charging Cost')
)


# Show the figure
fig.show()

# Export the Annualized Investment Costs plot to HTML
# fig.write_html("annual_costs.html")

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Load the data from the Excel file (make sure to use the correct path where the file is located)
voyage_df = df

# Plot for Annualized Investment Costs
fig_investment = go.Figure(data=[
    go.Bar(name='ICE Annual Cost', x=voyage_df['port_arrive'], y=voyage_df['annual_cost_ice'], marker=dict(color='darkred')),
    go.Bar(name='Battery Annual Cost', x=voyage_df['port_arrive'], y=voyage_df['annual_cost_bat'], marker=dict(color='darkblue'))
])

fig_investment.update_layout(
    barmode='group',
    title='Annualized Investment Costs for ICE and Battery Electric Ships by Port of Arrival',
    xaxis=dict(title='Port of Arrival'),
    yaxis=dict(title='Annualized Investment Cost (€)'),
    legend=dict(title='Cost Components')
)



# Display the plots
# fig_investment.show()

# Export the Annualized Investment Costs plot to HTML
fig_investment.write_html("annualized_investment_costs.html")



In [ ]:
# Plot for Operational Costs
fig_operational = go.Figure(data=[
    go.Bar(name='ICE Fuel Cost', x=voyage_df['port_arrive'], y=voyage_df['ice_fuel_cost'], marker=dict(color='salmon')),
    go.Bar(name='Battery Electric Cost', x=voyage_df['port_arrive'], y=voyage_df['be_electric_cost'], marker=dict(color='lightblue'))
])

fig_operational.update_layout(
    barmode='group',
    title='Operational Costs for ICE and Battery Electric Ships by Port of Arrival',
    xaxis=dict(title='Port of Arrival'),
    yaxis=dict(title='Operational Cost (€)'),
    legend=dict(title='Cost Components')
)

# Export the Operational Costs plot to HTML
fig_operational.write_html("operational_costs.html")